In [1]:
#Read in the extracted brown files
import glob

tagged_files = glob.glob("_extracted_brown/*.txt") #Read in the files and creates a list
print(type(tagged_files))
print(len(tagged_files)) #Should be 500

<class 'list'>
500


In [2]:
'''
Make the files into a list of a list of tuples
The tuple contains a str(word) and a set(tag(s)) 
Tag(s) because some words in the file contain more than one tag
'''
#I got help from the website where we download the extarcted brown text files
#https://kristopherkyle.github.io/Corpus-Linguistics-Working-Group/pos_tagging_1.html

#divide into sentences
full_data: list = []
for file in tagged_files:
    with open(file, 'r') as x:
        text = x.read().split("\n\n")
        for sent in text:
            sentence = []
            for word_line in sent.split("\n"):
                #Strip leading/trailing whitespace
                word_line = word_line.strip()
                
                #Skip empty lines
                if not word_line:
                    continue
                    
                # Check if split will work
                parts = word_line.split(" ", 1)
                if len(parts) != 2:
                    continue
                
                #Continue getting the word and tag(s)
                word_, pos = parts
                pos_set:set = set(pos.split("|"))
                sentence.append((word_, pos_set))
            
            if sentence:
                full_data.append(sentence)

In [3]:
#Better Sanity Check so I can see the structure
print(f"full_data type: {type(full_data)}")
print(f"Number of sentences: {len(full_data)}")

if full_data:
    first_sentence = full_data[0]
    print(f"First sentence type: {type(first_sentence)}")
    print(f"First sentence length: {len(first_sentence)}")
    
    if first_sentence:
        first_item = first_sentence[0]
        print(f"First item type: {type(first_item)}")
        print(f"First item: {first_item}")

full_data type: <class 'list'>
Number of sentences: 52108
First sentence type: <class 'list'>
First sentence length: 17
First item type: <class 'tuple'>
First item: ('In', {'IN'})


In [4]:
#HMM Model
import numpy as np
class HiddenMarkovModel:
    def __init__(self):
        #Initialize everything when I first create the Hidden Markov Model
        self.states = None
        self.observations = None
        
        #I need these states/observations to index
        #Because I need a way to calculate the probs (numpy understands integer indices, NOT strings!!!)
        self.states_to_idx = None
        self.states_to_idx = None
        
        #Make empty initial/tranmission/emission probabilities 
        #Since it's all learned during training
        self.initial_probs = None
        self.transition_probs = None
        self.emission_probs = None
        
    def train_HMM(self, training_data: list):
        """
        Trains the HMM on tagged data
        Calculates the initial, transmission, and emission probabilities
        Args:
            training_data (list): a list of a list of tuples with the words and POS tags
        """
        #Build the states and observations from the training data
        #Make them sets, since they don't allow duplication
        all_states = set()
        all_observations = set()
        for sentence in training_data:
            for word,tags in sentence:
                #Observations are based on the words
                all_observations.add(word)
                #The states are the tags
                all_states.update(tags)
        
        #Make the states and observations into lists
        self.states = list(all_states)
        self.observations = list(all_observations)
        
        #Make my state/observation index
        self.state_to_idx: dict = {state: i for i, state in enumerate(self.states)}
        self.obs_to_idx: dict = {obs: i for i, obs in enumerate(self.observations)}
        
        #initialize the empty matrices
        n_states = len(self.states)
        n_observations = len(self.observations)
        self.initial_probs = np.zeros(n_states)
        self.transition_probs = np.zeros((n_states, n_states))
        self.emission_probs = np.zeros((n_states, n_observations))
        
        #Now calculate the all the probabilities
        self.calculate_initial_probabilities(training_data)
        self.calculate_transition_probabilities(training_data)
        self.calculate_emission_probabilities(training_data)
        
        #DEBUGGING TO SEE IF IT WORKS PROPERLY
        #print("Sample transition probabilities:")
        #print(f"DT -> NN: {self.transition_probs[self.state_to_idx['DT']][self.state_to_idx['NN']]}")
        #print(f"NN -> VB: {self.transition_probs[self.state_to_idx['NN']][self.state_to_idx['VB']]}")

        #print("\nSample emission probabilities:")
        #print(f"P('The'|'DT'): {self.emission_probs[self.state_to_idx['DT']][self.obs_to_idx['The']]}")
        #print(f"P('cat'|'NN'): {self.emission_probs[self.state_to_idx['NN']][self.obs_to_idx['cat']]}")
        
    def calculate_initial_probabilities(self,training_data: list) -> np.ndarray:
        """
        Calculate the intial state probabilities P(tag|start)
        Args:
            training_data (list): a list of a list of tuples with the words and POS tags
        """
        for sentence in training_data:
            #Check to see if the sentence is empty
            if sentence:
                #Get the first words and tag(s) in the sentence
                first_word,first_tags = sentence[0]
                #Handle if the word has multiple tags
                for tag in first_tags:
                    #If the tag is in the state indec dictionary
                    if tag in self.state_to_idx:
                        tag_idx = self.state_to_idx[tag] #Forgot to add this and it lead to an error
                        #Fractional count if there's multiple tags
                        self.initial_probs[tag_idx] = self.initial_probs[tag_idx] + 1 / (len(first_tags))
    
    def calculate_transition_probabilities(self, training_data:list) -> np.ndarray:
        """
        Create the transition probability of current tag and previous tag
        P(tag i | tag i-1)
        Args:
            training_data (list): a list of a list of tuples with the words and POS tags
        """
        #Create a temporary matrix that will do all the calculations
        #Then store that into the self.transition_probability matrix
        transition_counts = np.zeros((len(self.states), len(self.states)))
        
        for sentence in training_data:
            #i in range of the entire sentence
            for i in range(1, len(sentence)):
                #Previous word and tags
                prev_word, prev_tags = sentence[i-1]
                #Current word and current tags
                current_word, current_tags = sentence[i]
                for previous_tag in prev_tags:
                    for current_tag in current_tags:
                        #If both the previous tag and the current tag are in the state index dicitonary
                        if previous_tag in self.state_to_idx and current_tag in self.state_to_idx:
                            prev_idx = self.state_to_idx[previous_tag]
                            curr_idx = self.state_to_idx[current_tag]
                            #Accidentally used + instead of *
                            transition_counts[prev_idx][curr_idx] +=  1 / (len(prev_tags) * len(current_tags))
                            
        #I need to normalize the transition matrix so it's between 0-1
        row_sums = transition_counts.sum(axis=1, keepdims=True)
        self.transition_probs = np.divide(transition_counts, row_sums, 
                                    out=np.zeros_like(transition_counts), 
                                    where=row_sums!=0)
    
    def calculate_emission_probabilities(self, training_data:list) -> np.ndarray:
        """
        Create the emission probability of the word and tag
        P(word | tag)
        Args:
            training_data (list): 
        """
        #Need a temporary matrix that does all the calculations
        #Then put it into the emission porbability matrix
        emission_counts = np.zeros((len(self.states), len(self.observations)))
        
        for sentence in training_data:
            for word, tags in sentence:
                if word in self.obs_to_idx:
                    word_idx = self.obs_to_idx[word]
                    for tag in tags:
                        if tag in self.state_to_idx:
                            tag_idx = self.state_to_idx[tag]
                            emission_counts[tag_idx][word_idx] += 1 / len(tags)
            
        #Normalize the counts into probabilities (I forgot this, which caused an issue in the code (It was more than 1))
        row_sums = emission_counts.sum(axis=1, keepdims=True)
        self.emission_probs = np.divide(emission_counts, row_sums,
                                    out=np.zeros_like(emission_counts),
                                    where=row_sums!=0)
        
    def viterbi(self, sentence: list) -> np.ndarray:
        """ My implementation of the viterbi algorithm from the textbook
        It returns the best path from the end of the sentence to the beginning
        Args:
            Sentence (list): a list of words
        """
        #Debug to see how the input is
        print(f"Input sentence: {sentence}")
     
        #Intialize the viterbi matrix and the bacpointer matrix
        viterbi = np.zeros((len(sentence), len(self.states)))
        backpointer = np.empty((len(sentence), len(self.states)))
       
        #for each state s from 1 to s
        first_word = sentence[0]
        for state_idx in range(len(self.states)):
            #make a viterbi matrix where viterbi[s][1] <- init_prob of that state * emission[state][observation[0]]
            #This is if the word is known
            if first_word in self.obs_to_idx:
                word_idx = self.obs_to_idx[first_word]
                #viterbi[first word][state] = initial prob of that state * emission[first word in the sentence]
                viterbi[0][state_idx] = self.initial_probs[state_idx] * self.emission_probs[state_idx][word_idx]
            
            #I need a way to handle unknown words
            else:
                #If the word is not known, make it 0
                viterbi[0][state_idx] = 0
            
            #Backpointer for the first word. There's no previous word so make it something to denote that
            backpointer[0][state_idx] = -1
            
            #Debugging statement to see what the initial viterbi row looks like
            #print(f"Initial viterbi row: {viterbi[0]}")
            
        #Going through my sentence (after the first word)
        for t in range(1, len(sentence)):
            #Get the index of the current word
            current_word = sentence[t]
            #See if the current word's index exists
            current_word_idx = self.obs_to_idx.get(current_word)
            
            #Go through every state besides the first word
            for current_state in range(len(self.states)):
                #Need variables to find which previous states gives us the max probability
                max_prob = -1
                best_prev_state = -1
                #Need to go through the previous states
                for prev_state in range(len(self.states)):
                    #The probability of the viterbi[previous word][previous state] * transition probability matrix[previous state][current state] * emission probability matrix[current state][word index]
                    prob = viterbi[t-1][prev_state] * self.transition_probs[prev_state][current_state] * self.emission_probs[current_state][current_word_idx]
                    
                    if prob > max_prob:
                        max_prob = prob #make the current probability the new max probability
                        best_prev_state = prev_state #make the current previous state the best previous state
                
                #After checking all the previous states, store the max probability adn the best previous state
                #Into the viterbi and the backpointer prespectively        
                viterbi[t][current_state] = max_prob
                
                # Debug statement to see what viterbi looks like after each time step
                #print(f"Viterbi at time {t}: {viterbi[t]}")
                
                backpointer[t][current_state] = best_prev_state
                         
        #Backtracking now
        #Get the last word of the sentence
        last_word = len(sentence) - 1
        #Get the best state for the last word with the argmax of the viterbi matrix
        best_last_state = np.argmax(viterbi[last_word])
        #Make a best path array with type int
        bestpath = np.zeros(len(sentence), dtype=int)
        #Make the best path of the last word the best last state
        bestpath[last_word] = best_last_state
        #Start from the second to last word and end at the beginning of the sentence
        #n-2, n-3, ..., 0
        for t in range(len(sentence)-2, -1, -1):
            bestpath[t] = backpointer[t+1][bestpath[t+1]]
            
        #Return the best path and the best path's probability
        return bestpath
    
    def predict(self, sentence: list) -> list:
        """
        Predict the part-of-speech tags for each word in the sentence
        Args:
            sentence (list): a list of words the HMM predicts
        Returns:
            a list of tuples (word, and predicted tag)
        """
        #Use the viterbi function
        tag_indices = self.viterbi(sentence)
        
        #Convert indices to actual tag names
        predicted_tags = [self.states[idx] for idx in tag_indices]
        
        #Pair words with predicted tags
        return list(zip(sentence, predicted_tags))
        

In [5]:
#Send in my list to train the model
hmm = HiddenMarkovModel()
hmm.train_HMM(full_data)

In [6]:
#A sample test Set for the HMM
#A few short sentences
test_sentence1 = ["The", "cat", "sat"]
test_sentence2 = ["Mark", "will", "pay", "the", "bill", "soon"]
test_sentence3 = ["I", "know", "how", "watch", "after", "a", "dog"]
test_sentence4 = ["I", "am", "so", "tired", "."]

#A two long ones
test_sentence_long = ["The", "police", "department", "said", "that", "the", "suspect", "has", "been", "apprehended", "today", ",", "they", "hope", "justice", "will", "be", "served", "."]
test_sentence_long2 = ["Today", "the", "studio", "announced", "that", "the", "new", "film", "will", "be", "about", "a", "girl", "who", "is", "transported", "to", "another", "world", "."]

predicted_tags1 = hmm.predict(test_sentence1)
print("HMM prediction of first sentence: ", predicted_tags1)
#Originally: HMM prediction of first sentence:  [('The', 'NPS'), ('cat', 'NPS'), ('sat', 'NPS')] - predicted it as NPs for some reason (Error with probability matrices)
#Fixed it issue: HMM prediction of first sentence:  [('The', 'DT'), ('cat', 'NN'), ('sat', 'VBD')]

predicted_tags2 = hmm.predict(test_sentence2)
print("HMM prediction of second sentence: ", predicted_tags2)
#HMM prediction of second sentence:  [('Mark', 'NNP'), ('will', 'MD'), ('pay', 'VB'), ('the', 'DT'), ('bill', 'NN'), ('soon', 'RB')]

predicted_tags3 = hmm.predict(test_sentence3)
print("HMM prediction of third sentence: ", predicted_tags3)
#HMM prediction of third sentence:  [('I', 'PRP'), ('know', 'VBP'), ('how', 'WRB'), ('watch', 'NN'), ('after', 'IN'), ('a', 'DT'), ('dog', 'NN')]

predicted_tags4 = hmm.predict(test_sentence4)
print("HMM prediction of fourth sentence: ", predicted_tags4)
#HMM prediction of fourth sentence:  [('I', 'PRP'), ('am', 'VBP'), ('so', 'RB'), ('tired', 'VBN'), ('.', '.')]

predicted_long_tags1 = hmm.predict(test_sentence_long)
print("HMM prediction of first long sentence: ", predicted_long_tags1)
#HMM prediction of first long sentence:  [('The', 'DT'), ('police', 'NN'), ('department', 'NN'), ('said', 'VBD'), ('that', 'IN'), ('the', 'DT'), ('suspect', 'NN'), ('has', 'VBZ'), ('been', 'VBN'), ('apprehended', 'VBN'), ('today', 'RB'), (',', ','), ('they', 'PRP'), ('hope', 'VBP'), ('justice', 'NN'), ('will', 'MD'), ('be', 'VB'), ('served', 'VBN'), ('.', '.')]

predicted_long_tags2 = hmm.predict(test_sentence_long2)
print("HMM prediction of second long sentence: ", predicted_long_tags2)
#HMM prediction of second long sentence:  [('Today', 'RB'), ('the', 'DT'), ('studio', 'NN'), ('announced', 'VBD'), ('that', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('film', 'NN'), ('will', 'MD'), ('be', 'VB'), ('about', 'IN'), ('a', 'DT'), ('girl', 'NN'), ('who', 'WP'), ('is', 'VBZ'), ('transported', 'VBN'), ('to', 'TO'), ('another', 'DT'), ('world', 'NN'), ('.', '.')]

possible_unknown1 = ["Computer", "science", "is", "cool", "but", "very", "hard", "."]
predicted_possible_unknown1 = hmm.predict(possible_unknown1)
print("HMM prediction of second long sentence: ", predicted_possible_unknown1)

Input sentence: ['The', 'cat', 'sat']
HMM prediction of first sentence:  [('The', 'DT'), ('cat', 'NN'), ('sat', 'VBD')]
Input sentence: ['Mark', 'will', 'pay', 'the', 'bill', 'soon']
HMM prediction of second sentence:  [('Mark', 'NNP'), ('will', 'MD'), ('pay', 'VB'), ('the', 'DT'), ('bill', 'NN'), ('soon', 'RB')]
Input sentence: ['I', 'know', 'how', 'watch', 'after', 'a', 'dog']
HMM prediction of third sentence:  [('I', 'PRP'), ('know', 'VBP'), ('how', 'WRB'), ('watch', 'NN'), ('after', 'IN'), ('a', 'DT'), ('dog', 'NN')]
Input sentence: ['I', 'am', 'so', 'tired', '.']
HMM prediction of fourth sentence:  [('I', 'PRP'), ('am', 'VBP'), ('so', 'RB'), ('tired', 'VBN'), ('.', '.')]
Input sentence: ['The', 'police', 'department', 'said', 'that', 'the', 'suspect', 'has', 'been', 'apprehended', 'today', ',', 'they', 'hope', 'justice', 'will', 'be', 'served', '.']
HMM prediction of first long sentence:  [('The', 'DT'), ('police', 'NN'), ('department', 'NN'), ('said', 'VBD'), ('that', 'IN'), ('th

I created these sentences with tags via this website:
https://parts-of-speech.info/
And this is what the website predicted for each sentence
For sentence 1: [DT, NN, VBD] - Matches the HMM Model's prediction

For sentence 2: [NNP, MD, VB, DT, NN, RB] - Matches the HMM's prediction

For sentence 3: [PRP, VBP, WRB, VB, DT, NN] - Matches the HMM's prediction

For sentence 4: [PRP, VBP, RB, JJ] - Matches the HMM's prediction except for the punctuation

For long sentence 1: [DT, NN, NN, VBD, IN, DT, NN, VBZ, VBN, VBN, NN, PRP, VBP, NN, MD, VB, VBN] - Matches the HMM's prediction except for the punctuations

For long sentence 2: [NN, DT, NN, VBD, IN, DT, JJ, NN, MD, VB, IN, DT, NN, WP, VBZ, VBN, TO, DT, NN] - The model predicted that Today was an RB and it had TO as a tag unlike the website

For possible unknown sentence 1: Here everything is tagegd as the same.


In [7]:
#Feature engineering for the extracted brown files
#I got help from the geeksforgeeks website
#https://www.geeksforgeeks.org/nlp/conditional-random-fields-crfs-for-pos-tagging-in-nlp/
def word_features(sentence, i):
    word = sentence[i][0]
    pos_tag = sentence[i][1]
    features = {
        'word': word,
        'pos' : pos_tag,
        'is_first': i == 0, #if the word is a first word
        'is_last': i == len(sentence) - 1,  #if the word is a last word
        'is_capitalized': word[0].upper() == word[0],
        'is_all_caps': word.upper() == word,      #word is in uppercase
        'is_all_lower': word.lower() == word,      #word is in lowercase
         #prefix of the word
        'prefix-1': word[0],   
        'prefix-2': word[:2],
        'prefix-3': word[:3],
         #suffix of the word
        'suffix-1': word[-1],
        'suffix-2': word[-2:],
        'suffix-3': word[-3:],
         #extracting previous word
        'prev_word': '' if i == 0 else sentence[i-1][0],
         #extracting next word
        'next_word': '' if i == len(sentence)-1 else sentence[i+1][0],
        'prev_pos': '' if i == 0 else sentence[i-1][1],  # Previous word's POS tag
        'next_pos': '' if i == len(sentence)-1 else sentence[i+1][1],  # Next word's POS tag
        'has_hyphen': '-' in word,    #if word has hypen
        'is_numeric': word.isdigit(),  #if word is in numeric
        'capitals_inside': word[1:].lower() != word[1:]
    }
    return features

In [8]:
X = []
y = []
for sentence in full_data:
    X_sentence = []
    y_sentence = []
    #Go through every sentence in the full data list
    for i in range(len(sentence)):
        #Append the word features into the X_sentence
        #print(f"Sentence[i][0]: {sentence[i][0]}") 
        X_sentence.append(word_features(sentence,i))
        #print(f"Sentence[i][1] is: {sentence[i][1]}")
        y_sentence.append(sentence[i][1])
        
    #Append the sentences into the original list
    X.append(X_sentence)
    y.append(y_sentence)
    
#Split the extracted files (80% training, 20% testing)
split = int(0.8 * len(X))
#Get every word,tag up to 80% of the orignal X and y
X_train = X[:split]
y_train = y[:split]
#Get the remaining 20% of the original X and y
X_test = X[split:]
y_test = y[split:]

In [9]:
#check the size of the training and test sets
print(f"The length of the x_train is : {len(X_train)}")
print(f"The length of the y_train is : {len(y_train)}")
print(f"The length of the X_test is : {len(X_test)}")
print(f"The length of the y_test is : {len(y_test)}")

The length of the x_train is : 41686
The length of the y_train is : 41686
The length of the X_test is : 10422
The length of the y_test is : 10422


In [10]:
#CRF
import time
import random
import math
from collections import defaultdict

class LinearChainConditionalRandomField:
    def __init__(self, learning_rate=0.1, max_iter=15, batch_size=500, l2_penalty=0.01):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.l2_penalty = l2_penalty
        self.weights = defaultdict(float)
        self.all_labels = set()
        self.label_list = []  # Cache for faster iteration
    
    def convert_features(self, features_dict, label):
        """Converts features
            Args:
                features_dict (dict): a sictionary of all the features I pass in from training/test set
                label (str): a tag associated with the word
            Returns:
                dictionary: a dictionary of important features
        """
        feature_vector = {}
        
        # ONLY these 3 features - remove the rest
        word = features_dict['word']
        feature_vector[f"w_{word}_{label}"] = 1
        
        if features_dict['prev_word']:
            feature_vector[f"p_{features_dict['prev_word']}_{label}"] = 1
            
        if features_dict.get('is_capitalized'):
            feature_vector[f"c_{label}"] = 1
            
        return feature_vector
    
    def compute_score(self, sequence_features, labels):
        """Optimized scoring - direct dict lookups
            Args:
                sequence_features(list): the features of the words in a sequence/sentence
                labels (list): All the tags for a sequence/sentence
            Returns:
                float: the score of the weights for word and its labels, the previous word and its labels as well its features
        """
        score = 0.0
        prev_label = None
        
        for features, label in zip(sequence_features, labels):
            # Direct dict lookups - no function calls
            word = features['word']
            score += self.weights.get(f"w_{word}_{label}", 0)
            
            if features['prev_word']:
                score += self.weights.get(f"p_{features['prev_word']}_{label}", 0)
                
            if features.get('is_capitalized'):
                score += self.weights.get(f"c_{label}", 0)
            
            if prev_label is not None:
                score += self.weights.get(f"t_{prev_label}_{label}", 0)
            
            prev_label = label
            
        return score
    
    def approximate_forward_pass(self, sequence_features):
        """
        APPROXIMATE forward pass
        Uses unary potentials only, ignores transitions for gradient
        Args:
            sequence_features(list): the features of words in a sentence
        Returns:
            float: the total partitiion function (as a log)
        """
        T = len(sequence_features)
        if T == 0:
            return 0.0
        
        labels = self.label_list
        total_log_Z = 0.0
        
        # Sum over positions (assumes independence - fast approximation)
        for t in range(T):
            scores = []
            for label in labels:
                word = sequence_features[t]['word']
                score = self.weights.get(f"w_{word}_{label}", 0)
                if sequence_features[t].get('is_capitalized'):
                    score += self.weights.get(f"c_{label}", 0)
                scores.append(score)
            
            if scores:
                max_score = max(scores)
                total_log_Z += max_score + math.log(sum(math.exp(s - max_score) for s in scores))
        
        return total_log_Z
    
    def compute_batch_gradient_fast(self, X_batch, y_batch):
        """ULTRA-FAST gradient computation
            Args:
            Returns:
                dict: a dictionary of all the batch gradients
                float: the loss from each batch
        """
        grad = defaultdict(float)
        batch_loss = 0.0
        
        for seq_features, true_labels in zip(X_batch, y_batch):
            # Compute true score (fast)
            true_score = self.compute_score(seq_features, true_labels)
            
            # Compute approximate log_Z (very fast)
            log_Z = self.approximate_forward_pass(seq_features)
            
            if math.isfinite(log_Z):
                batch_loss -= (true_score - log_Z)
                
                # Boost true features (perceptron-style)
                prev_label = None
                for features, label in zip(seq_features, true_labels):
                    # Emission
                    word = features['word']
                    grad[f"w_{word}_{label}"] += self.learning_rate
                    
                    if features['prev_word']:
                        grad[f"p_{features['prev_word']}_{label}"] += self.learning_rate * 0.3
                        
                    if features.get('is_capitalized'):
                        grad[f"c_{label}"] += self.learning_rate * 0.1
                    
                    # Transition
                    if prev_label is not None:
                        grad[f"t_{prev_label}_{label}"] += self.learning_rate * 0.5
                    
                    prev_label = label
        
        return grad, batch_loss
    
    def fit(self, X_train, y_train):
        """Training function-optimized
            Args:
                X_train(list): a list of all the words in the training data
                y_train(list): a list of the tag(s) for eahc word in the training data
        """
        # Convert labels
        y_train_single = []
        for sentence_labels in y_train:
            sentence_single = [next(iter(tag_set)) for tag_set in sentence_labels]
            y_train_single.append(sentence_single)
            self.all_labels.update(sentence_single)
        
        self.label_list = list(self.all_labels)  # Cache for speed
        
        print(f"FAST training on {len(X_train)} sequences")
        print("Using approximate gradients")
        
        for iteration in range(self.max_iter):
            start_time = time.time()
            total_loss = 0.0
            
            # Shuffle
            indices = list(range(len(X_train)))
            random.shuffle(indices)
            X_shuffled = [X_train[i] for i in indices]
            y_shuffled = [y_train_single[i] for i in indices]
            
            # Process batches
            for batch_start in range(0, len(X_shuffled), self.batch_size):
                batch_end = min(batch_start + self.batch_size, len(X_shuffled))
                X_batch = X_shuffled[batch_start:batch_end]
                y_batch = y_shuffled[batch_start:batch_end]
                
                # FAST gradient computation
                grad, batch_loss = self.compute_batch_gradient_fast(X_batch, y_batch)
                total_loss += batch_loss
                
                # Update weights
                for feat, update in grad.items():
                    # Simple update - skip complex regularization during training
                    self.weights[feat] += update
                    
                    # Optional: lightweight clipping
                    if abs(self.weights[feat]) > 20.0:
                        self.weights[feat] = math.copysign(20.0, self.weights[feat])
            
            avg_loss = total_loss / len(X_train)
            epoch_time = time.time() - start_time
            
            print(f"Iter {iteration}, Loss: {avg_loss:.4f}, Time: {epoch_time:.1f}s")
            
            # Learning rate decay
            self.learning_rate *= 0.9
    
    def viterbi_decode(self, sequence_features):
        """Optimized Viterbi
            Args:
                sequence_features(list): the features of every word in a sequence
            Returns:
                list: a list of the best tags for words all the way from the end of a sentence to the beginning
        """
        T = len(sequence_features)
        if T == 0:
            return []
        
        labels = self.label_list
        delta = [defaultdict(float) for _ in range(T)]
        psi = [defaultdict(str) for _ in range(T)]
        
        # Initialize
        for label in labels:
            features = sequence_features[0]
            word = features['word']
            delta[0][label] = self.weights.get(f"w_{word}_{label}", 0)
            if features.get('is_capitalized'):
                delta[0][label] += self.weights.get(f"c_{label}", 0)
            psi[0][label] = None
        
        # Recursion
        for t in range(1, T):
            features = sequence_features[t]
            word = features['word']
            
            for current_label in labels:
                best_score = -1e10
                best_prev_label = None
                
                # Precompute emission once per (t, current_label)
                emission_score = self.weights.get(f"w_{word}_{current_label}", 0)
                if features.get('is_capitalized'):
                    emission_score += self.weights.get(f"c_{current_label}", 0)
                
                for prev_label in labels:
                    transition_score = self.weights.get(f"t_{prev_label}_{current_label}", 0)
                    score = delta[t-1][prev_label] + emission_score + transition_score
                    
                    if score > best_score:
                        best_score = score
                        best_prev_label = prev_label
                
                delta[t][current_label] = best_score
                psi[t][current_label] = best_prev_label
        
        # Backtrack
        best_path = [None] * T
        best_score = -1e10
        
        for label in labels:
            if delta[T-1][label] > best_score:
                best_score = delta[T-1][label]
                best_path[T-1] = label
        
        for t in range(T-2, -1, -1):
            best_path[t] = psi[t+1][best_path[t+1]]
        
        return best_path
    
    def evaluate(self, X_test, y_test):
        """Fast evaluation"""
        correct = 0
        total = 0
        
        for i, (seq_features, true_seq) in enumerate(zip(X_test, y_test)):
            if i >= 200:  # Limit evaluation for speed
                break
                
            pred_labels = self.viterbi_decode(seq_features)
            for pred_label, true_set in zip(pred_labels, true_seq):
                if pred_label in true_set:
                    correct += 1
                total += 1
        
        return correct / total if total > 0 else 0


In [11]:

# Initialize with optimized parameters
crf = LinearChainConditionalRandomField(
    learning_rate=0.05,
    max_iter=10,
    batch_size=2000,  # Process 2000 sequences at once
    l2_penalty=0.01
)

# Train on full dataset
print("=== FULL DATASET TRAINING ===")
crf.fit(
    X_train, 
    y_train,
)

# Final evaluation
print("\n=== FINAL EVALUATION ===")
test_accuracy = crf.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


=== FULL DATASET TRAINING ===
FAST training on 41686 sequences
Using approximate gradients
Iter 0, Loss: -395.7195, Time: 14.7s
Iter 1, Loss: -540.5592, Time: 15.2s
Iter 2, Loss: -573.1979, Time: 15.6s
Iter 3, Loss: -590.2288, Time: 14.6s
Iter 4, Loss: -600.7107, Time: 13.8s
Iter 5, Loss: -608.0966, Time: 14.1s
Iter 6, Loss: -613.8068, Time: 15.1s
Iter 7, Loss: -618.3808, Time: 14.9s
Iter 8, Loss: -622.1357, Time: 14.4s
Iter 9, Loss: -625.2462, Time: 13.6s

=== FINAL EVALUATION ===
Test Accuracy: 0.8795


In [12]:
#Read in the GMB dataset
import pandas as pd
#Read it in without headers with latin1 encoding
gmb_pd = pd.read_csv("./GMB_dataset.txt", sep="\t", header=None,encoding="latin1")
#Take the first row as the heading with the names in the file
#Re-index the dataset appropriately
gmb_pd.columns = gmb_pd.iloc[0]
gmb_pd = gmb_pd[1:]
gmb_pd.columns = ['Index','Sentence #','Word','POS','Tag']
gmb_pd = gmb_pd.reset_index(drop=True)
gmb_pd.head()

,Index,Sentence #,Word,POS,Tag
0,0.0,1.0,Thousands,NNS,O
1,1.0,1.0,of,IN,O
2,2.0,1.0,demonstrators,NNS,O
3,3.0,1.0,have,VBP,O
4,4.0,1.0,marched,VBN,O


In [13]:
#get the sentences from the dataset (from kaggle)
#A class to retrieve the sentences from the dataset
class getsentence(object):
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        #using pandas groupby to get every sentence(with the word, pos, and tag)
        self.grouped = self.data.groupby("Sentence #")[['Word', 'POS', 'Tag']].apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [14]:
getter = getsentence(gmb_pd)
sentences = getter.sentences
print(len(sentences))
#This is how a sentence will look like. 
print(sentences[0])

2999
[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [15]:
import nltk
from nltk.corpus import gazetteers #This is good for locations (albeit it's more US focused)
# Load gazetteer words into a set for O(1) lookups
gazetteer_words = set()
# The gazetteers corpus contains files for different countries
for fileid in gazetteers.fileids():
    words = gazetteers.words(fileid)
    gazetteer_words.update(word.lower() for word in words)

print(f"Loaded {len(gazetteer_words)} gazetteer words")

Loaded 995 gazetteer words


In [16]:
#Feature engineering necessary for the CRF
#Modified for Named Entity Recognition (Looks at the previous word and the next word in the sentence)
def word2features(sent, i):
    """
    Converts a word into a list of features indices

    Args:
        sent (list): a list of words with their features
        i (int): the number to get a word or tag in a sentence

    Returns:
        list: a list of features that belong to a word
    """
    word = sent[i][0]
    postag = sent[i][1]
    word_lower = word.lower()
    
    features = []
    
    # Basic word features - now as indices instead of dictionaries
    features.append(f"word.lower():{word_lower}")
    features.append(f"word.istitle():{word.istitle()}")
    features.append(f"word.isupper():{word.isupper()}")
    features.append(f"postag:{postag}")
    features.append(f"postag[:2]:{postag[:2]}")
    
    in_gazetteer = word_lower in gazetteer_words
    features.append(f"in_gazetteer:{in_gazetteer}")
    
    # Gazetteer + word shape combinations
    if in_gazetteer:
        features.append("gazetteer_istitle:True" if word.istitle() else "gazetteer_istitle:False")
        features.append("gazetteer_isupper:True" if word.isupper() else "gazetteer_isupper:False")
        features.append("gazetteer_all_lower:True" if word.islower() else "gazetteer_all_lower:False")
        
        # Gazetteer + position in sentence
        if i == 0:
            features.append("gazetteer_sentence_start:True")
        if i == len(sent)-1:
            features.append("gazetteer_sentence_end:True")
            
    # Context features gazetteer
    if i > 0:
        prev_word = sent[i-1][0]
        prev_lower = prev_word.lower()
        prev_postag = sent[i-1][1]
        
        features.append(f"-1:word.lower():{prev_lower}")
        features.append(f"-1:postag:{prev_postag}")
        
        # Previous word gazetteer features
        prev_in_gazetteer = prev_lower in gazetteer_words
        features.append(f"-1:in_gazetteer:{prev_in_gazetteer}")
        
        # Combined features: current AND previous in gazetteer
        if in_gazetteer and prev_in_gazetteer:
            features.append("both_current_prev_gazetteer:True")
    
    #If it's the first word, add the BOS tag        
    else:
        features.append("BOS")
    
    #Look for the next word    
    if i < len(sent)-1:
        next_word = sent[i+1][0]
        next_lower = next_word.lower()
        next_postag = sent[i+1][1]
        
        features.append(f"+1:word.lower():{next_lower}")
        features.append(f"+1:postag:{next_postag}")
        
        # Next word gazetteer features
        next_in_gazetteer = next_lower in gazetteer_words
        features.append(f"+1:in_gazetteer:{next_in_gazetteer}")
        
        # Combined features: current AND next in gazetteer
        if in_gazetteer and next_in_gazetteer:
            features.append("both_current_next_gazetteer:True")
    
    #If it's the end, add EOD feature
    else:
        features.append("EOS")
    
    return features

In [17]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [ ]:
#Split dataset into training and testing
import sklearn
from sklearn.model_selection import train_test_split

#I need to split the sentences for the CRF
# Split into train and test sets (80/20)
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)
print(f"Training: {len(train_sentences)} sentences")

print(f"Test: {len(test_sentences)} sentences")

Training: 2399 sentences
Test: 600 sentences


In [ ]:
#CRF for Named Entity Recognition (NER)
import numpy as np
from collections import defaultdict
class ConditionalRandomFieldNer:
    """
    My implementation of a linear chain conditional random field
    for named entity recognition
    Did not finish in time
    """
    def __init__(self):
        self.feature_weights = defaultdict(float)
        self.labels = set() #A set of all the labels (no duplicates)
        self.feature_map = {} #A map of every feature {feature_string -> feature_index}
        self.label_map = {} #A map of every label {label string -> label_index}
    
    def build_feature_maps(self, training_data):
        """
        Build feature maps with labels

        Args:
            training_data (_type_): _description_
        """
        all_features  = set()
        for sentence in training_data:
            #Collect the features and labels
            features_seq = sent2features(sentence)
            for feature_list in features_seq:
                for feature_str in feature_list:
                    all_features.add(feature_str)
                    
        
        # Create mappings for the later functions
        self.feature_map = {feat: idx for idx, feat in enumerate(sorted(all_features))}
        self.label_map = {label: idx for idx, label in enumerate(sorted(self.labels))}
        
        print(f"Built feature map with {len(self.feature_map)} features")
        print(f"Labels: {list(self.label_map.keys())}")
    def convert_features(self, feature_list, current_label):
        """
        Convert feature list to feature vector for a specific label
        Args:
            feature_list (list):
            current_label(str)
        Returns:
        
        """
        # This creates emission features: feature + label combinations
        feature_vector = defaultdict(float)
        
        for feature_str in feature_list:
            # Emission feature: feature + current_label
            emission_feature = f"{feature_str}|||{current_label}"
            feature_vector[emission_feature] += 1.0
            
            # Also include the base feature (without label)
            feature_vector[feature_str] += 1.0
        
        return feature_vector
    
    def compute_transition_features(self, prev_label, current_label):
        """Create transition features between labels"""
        transition_feature = f"TRANSITION:{prev_label}→{current_label}"
        return {transition_feature: 1.0}
    
    def compute_sequence_score(self, features_sequence, labels_sequence):
        """Compute score for a given feature sequence and label sequence"""
        total_score = 0.0
        
        for t in range(len(features_sequence)):
            # Emission score
            emission_features = self.convert_features(features_sequence[t], labels_sequence[t])
            emission_score = sum(self.feature_weights[feat] * value 
                               for feat, value in emission_features.items())
            total_score += emission_score
            
            # Transition score (except for first position)
            if t > 0:
                transition_features = self.compute_transition_features(
                    labels_sequence[t-1], labels_sequence[t]
                )
                transition_score = sum(self.feature_weights[feat] * value 
                                     for feat, value in transition_features.items())
                total_score += transition_score
        
        return total_score
    
    def viterbi_decode(self,features_sequence, possible_labels):
        """Viterbi algorithm for finding the most likely label sequence"""
        T = len(features_sequence)
        
        # Initialize delta and psi
        delta = [defaultdict(lambda: -float('inf')) for _ in range(T)]
        psi = [defaultdict(str) for _ in range(T)]
        
        # Initialize first position
        for label in possible_labels:
            # Only emission features for first position (no transition)
            emission_features = self.convert_features(features_sequence[0], label)
            delta[0][label] = sum(self.feature_weights[feat] * value 
                                for feat, value in emission_features.items())
            psi[0][label] = ""  # No previous label
        
        # Recursion
        for t in range(1, T):
            for current_label in possible_labels:
                best_score = -float('inf')
                best_prev_label = None
                
                for prev_label in possible_labels:
                    # Emission features for current position
                    emission_features = self.convert_features(features_sequence[t], current_label)
                    emission_score = sum(self.feature_weights[feat] * value 
                                       for feat, value in emission_features.items())
                    
                    # Transition features
                    transition_features = self.compute_transition_features(prev_label, current_label)
                    transition_score = sum(self.feature_weights[feat] * value 
                                         for feat, value in transition_features.items())
                    
                    score = delta[t-1][prev_label] + emission_score + transition_score
                    
                    if score > best_score:
                        best_score = score
                        best_prev_label = prev_label
                
                delta[t][current_label] = best_score
                psi[t][current_label] = best_prev_label
        
        # Backtrack
        best_path = [None] * T
        best_score = -float('inf')
        
        # Find best final label
        for label in possible_labels:
            if delta[T-1][label] > best_score:
                best_score = delta[T-1][label]
                best_path[T-1] = label
        
        # Backtrack through the sequence
        for t in range(T-2, -1, -1):
            best_path[t] = psi[t+1][best_path[t+1]]
        
        return best_path, best_score
    def process_sentence():
        loss = 0.0
        return loss, gradient
    def fit(self, training_data, iterations=20, learning_rate=0.01):
        """
        This handles the training of the CRF for NER
        Args:
            training_data (list): a list of a list of sentences
            iterations (int, optional): the number of iterations for training. Defaults to 20.
            learning_rate (float, optional): learning rate of the model. Defaults to 0.01.
        """
        print("Building feature maps...")
        self.build_feature_maps(training_data)
        
        print(f"Starting training with {len(training_data)} sentences...")
        for iteration in range(iterations):
            total_loss = 0.0
            for sentence in training_data:
                loss = self.process_sentence(sentence)
                total_loss += loss
            
        print(f"Iteration {iteration}, Loss: {total_loss:.4f}")
        #Learning rate decay
        self.learning_rate *= 0.95
    def predict(self, sentence):
        """Predict labels for a sentence"""
        features_sequence = sent2features(sentence)
        possible_labels = list(self.label_map.keys())  # All possible labels
        
        best_path, best_score = self.viterbi_decode(features_sequence, possible_labels)
        return best_path
    
    def model_evaluation(self, X_test, y_test):
        correct_predictions = 0
        total_predictions = 0
        
        all_true_labels = []
        all_predicted_labels = []
        
        for sentence in test_sentences:
            true_labels = sent2labels(sentence)
            predicted_labels = self.predict(sentence)
            
            all_true_labels.extend(true_labels)
            all_predicted_labels.extend(predicted_labels)
            
            correct_predictions += sum(1 for t, p in zip(true_labels, predicted_labels) if t == p)
            total_predictions += len(true_labels)
        
        accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
        #Something to work on
        precision = accuracy
        recall = accuracy
        f1_score = (2 *precision * recall) / (precision + recall)
        return precision, recall, accuracy, f1_score

In [ ]:
#Now train the model with the GMB dataset
crf = ConditionalRandomFieldNer()
crf.fit()
#Return the model evaluation
model_precision, model_recall, model_accuracy, model_f1_score = crf.model_evaluation()

#Print out the model's evaluation
print(f"Model Precision: {model_precision * 100:.2f}"
      f"Model Recall: {model_recall * 100:.2f}"
      f"Model Accuracy: {model_accuracy * 100:.2f}"
      f"Model's F1_score: {model_f1_score * 100:.2f}")